<a href="https://colab.research.google.com/github/FeelOneE/price_prediction/blob/main/coin_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/스터디/코인 트레이드/data


/content/drive/MyDrive/스터디/코인 트레이드/data


In [3]:
%ls -al

total 23809
-rw------- 1 root root   69784 Dec 19 13:56 cnn.h5
drwx------ 4 root root    4096 Dec 19 09:41 cnn_prediction/
-rw------- 1 root root  185256 Dec 20 07:40 cnn_prediction_bora.h5
-rw------- 1 root root  185352 Dec 20 08:06 cnn_prediction_eos.h5
-rw------- 1 root root 4681048 Dec 26 13:37 cnn_prediction_sand.h5
-rw------- 1 root root  185256 Dec 20 07:55 cnn_prediction_xlm.h5
-rw------- 1 root root 3802325 Dec 20 07:35 KRW-BORA.csv
-rw------- 1 root root 3832322 Dec 20 08:03 KRW-EOS.csv
-rw------- 1 root root 3836665 Dec 20 07:10 KRW-SAND.csv
-rw------- 1 root root 3740209 Dec 20 07:46 KRW-XLM.csv
-rw------- 1 root root    1349 Dec 20 07:36 scaler_bora.pkl
-rw------- 1 root root    1349 Dec 20 08:03 scaler_eos.pkl
-rw------- 1 root root    1349 Dec 19 09:40 scaler.pkl
-rw------- 1 root root    1349 Dec 20 07:16 scaler_sand.pkl
-rw------- 1 root root    1349 Dec 20 07:47 scaler_xlm.pkl
-rw------- 1 root root       0 Dec 19 07:57 train.csv
-rw------- 1 root root 3847495 Dec 18 

In [31]:
import pandas as pd

data = pd.read_csv("XRP.csv")
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,2021-12-18T00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,2021-12-18T00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,2021-12-17T23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,2021-12-17T23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,2021-12-17T23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [32]:
# 시간 T를 기준으로 나누어 날짜는 제외하고 시간 분만 남게 처리
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split('T', expand=True).apply(lambda x : x[1], axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [33]:
# 시간을 분으로 치환
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split(':', expand=True).apply(lambda x : (int(x[0])*60)+(int(x[1])) , axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,5,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,0,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,1435,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,1430,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,1425,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [34]:
# 안쓰는 컬럼 삭제 market, utc_time, unit, timestamp
data.drop(['market','candle_date_time_utc','unit','timestamp'], axis=1, inplace=True)
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,5,986.0,987.0,984.0,987.0,1.785798e+09,1.812305e+06
1,0,989.0,989.0,981.0,986.0,2.128017e+09,2.161762e+06
2,1435,984.0,989.0,983.0,989.0,2.275873e+09,2.311628e+06
3,1430,987.0,990.0,984.0,985.0,3.238981e+09,3.281353e+06
4,1425,983.0,987.0,981.0,987.0,5.052325e+09,5.141900e+06


In [35]:
# price, volume 소수점 5자리까지만 
data['candle_acc_trade_price'] = data.candle_acc_trade_price.apply(lambda x : '%.5f' % x)
data['candle_acc_trade_volume'] = data.candle_acc_trade_volume.apply(lambda x : '%.5f' % x)
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,5,986.0,987.0,984.0,987.0,1785797775.07049,1812305.45733
1,0,989.0,989.0,981.0,986.0,2128016988.39383,2161761.89739
2,1435,984.0,989.0,983.0,989.0,2275873191.28294,2311628.47649
3,1430,987.0,990.0,984.0,985.0,3238981389.51325,3281353.39430
4,1425,983.0,987.0,981.0,987.0,5052324608.41463,5141900.24199


In [36]:
# 시간순서대로 데이터 다시 정렬
data = data[::-1]
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
30199,1240,1475.0,1480.0,1470.0,1480.0,2449703432.48617,1661509.79377
30198,1245,1480.0,1480.0,1470.0,1480.0,1401303636.71963,949623.96941
30197,1250,1475.0,1480.0,1475.0,1480.0,964682189.53982,652588.80397
30196,1255,1480.0,1480.0,1475.0,1475.0,685879546.75352,464264.68260
30195,1260,1475.0,1480.0,1475.0,1480.0,519939889.71916,351858.39885


In [37]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
std_data = data.copy()
std_data[:] = scaler.fit_transform(data[:])
std_data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
30199,0.864111,0.784483,0.783237,0.803922,0.791667,0.022750,0.021775
30198,0.867596,0.791667,0.783237,0.803922,0.791667,0.013011,0.012443
30197,0.871080,0.784483,0.783237,0.810458,0.791667,0.008956,0.008549
30196,0.874564,0.791667,0.783237,0.810458,0.784483,0.006366,0.006081
30195,0.878049,0.784483,0.783237,0.810458,0.791667,0.004825,0.004607


In [38]:
# 학습된 정규화로 로우 데이터 정규화 수행
#data[:2]
scaler.transform(data[:2])

array([[0.8641115 , 0.78448276, 0.78323699, 0.80392157, 0.79166667,
        0.02274972, 0.02177474],
       [0.86759582, 0.79166667, 0.78323699, 0.80392157, 0.79166667,
        0.01301138, 0.01244297]])

In [ ]:
import joblib 
# 객체를 pickled binary file 형태로 저장한다 
file_name = 'scaler_eos.pkl' 
joblib.dump(scaler, file_name) 

['scaler_eos.pkl']

In [ ]:
import joblib 
# pickled binary file 형태로 저장된 객체를 로딩한다 
file_name = 'scaler.pkl' 
obj = joblib.load(file_name) 

In [ ]:
%ls -al

total 3759
-rw------- 1 root root    1349 Dec 19 09:40 scaler.pkl
-rw------- 1 root root       0 Dec 19 07:57 train.csv
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [39]:
# 정답 라벨 붙이기
# !-- 데이터를 60개로 묶고, 마지막 종가(trade_price)를 기준으로 
# !-- 다음 60개의 데이터 중 최대 값이 종가의 1.035배 이상인 적이 있는지 찾기

# 수익률을 반올림하여 소수점 두째 짜리 까지 라벨로 저장, 마지막에 1을 빼서 음수면 마이너스, 양수면 수익
import numpy as np

data_set = []
x_data = []
y_data = []
def find_label(index):
  tmp = []
  sample = data[index:index+120]
  next_sample = data[index+120:index+240]

  trade_price = sample['trade_price'].iloc[-1]
  
  #print(trade_price)

  price_list = []
  price_list.extend(next_sample['opening_price'])
  price_list.extend(next_sample['trade_price'])

  max_price = max(price_list)
  revenue_rate = round( (max_price / trade_price) - 1 ,2)
  
  #학습 데이터에 추가
  if revenue_rate <= 0.1 and revenue_rate >= 0 :
    y_data.append( revenue_rate * 100 )
    x_data.append(scaler.transform(sample))

  #print(max_price)

  #expect_price = trade_price*1.03
  #print(expect_price)
  

  #if max_price >= expect_price :
    #y_data.append(1)

    #return 1
  # print("trade_price : ",trade_price)
  
  # print("max_price :",max_price)
  # print("revenue_rate! : ", revenue_rate)
  # print("Y-data : ", round( revenue_rate - 1 ,2))
  # print('-'*30)
  
  #y_data.append(0)
  #return 0

In [40]:
# 전체 데이터와 정답 레이블의 비율
total_cnt = 0
cnt = 0
skip_size = 1
for idx in range(0, len(data)-240, skip_size):
  find_label(idx)
  #total_cnt += 1

#rate = float(cnt / total_cnt)
#print(rate)


In [41]:
df_label = pd.DataFrame(y_data)
print(df_label[0].value_counts())

1.0     8817
0.0     7918
2.0     5353
3.0     3427
4.0     1676
5.0      880
6.0      518
7.0      503
8.0      379
9.0      161
10.0     112
Name: 0, dtype: int64


In [15]:
import numpy as np
from keras.layers import Conv1D, Activation, MaxPool1D, Flatten, Dense,Dropout
from keras.models import Sequential
from keras.datasets import mnist
from keras.utils import np_utils

In [42]:
import numpy as np
x_data_np = np.array(x_data)
y_data_np = np_utils.to_categorical(np.array(y_data),num_classes=11)
x_data_np.shape, y_data_np.shape

((29744, 120, 7), (29744, 11))

In [43]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data_np, y_data_np, test_size=0.4, shuffle=True, random_state=1004 )

# 모델에 입력가능하게 차원을 축소(?)함
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
x_train.shape, x_test.shape

x_train.shape,x_test.shape,y_train.shape,y_test.shape

((17846, 120, 7, 1), (11898, 120, 7, 1), (17846, 11), (11898, 11))

In [44]:
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_timesteps, n_features, n_outputs 

(120, 7, 11)

In [45]:
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]

verbose, epochs, batch_size = 2, 100, 32

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 118, 64)           1408      
                                                                 
 conv1d_7 (Conv1D)           (None, 116, 64)           12352     
                                                                 
 dropout_3 (Dropout)         (None, 116, 64)           0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 58, 64)           0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 3712)              0         
                                                                 
 dense_6 (Dense)             (None, 100)               371300    
                                                      

In [46]:
# fit network
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=(x_test,y_test))

Epoch 1/100
558/558 - 15s - loss: 1.7713 - accuracy: 0.2908 - val_loss: 1.8134 - val_accuracy: 0.3006 - 15s/epoch - 26ms/step
Epoch 2/100
558/558 - 14s - loss: 1.7234 - accuracy: 0.3049 - val_loss: 1.7300 - val_accuracy: 0.3096 - 14s/epoch - 24ms/step
Epoch 3/100
558/558 - 14s - loss: 1.6851 - accuracy: 0.3176 - val_loss: 1.7171 - val_accuracy: 0.3197 - 14s/epoch - 24ms/step
Epoch 4/100
558/558 - 14s - loss: 1.6488 - accuracy: 0.3349 - val_loss: 1.6820 - val_accuracy: 0.3359 - 14s/epoch - 24ms/step
Epoch 5/100
558/558 - 13s - loss: 1.6027 - accuracy: 0.3515 - val_loss: 1.6923 - val_accuracy: 0.3590 - 13s/epoch - 24ms/step
Epoch 6/100
558/558 - 14s - loss: 1.5500 - accuracy: 0.3748 - val_loss: 1.6420 - val_accuracy: 0.3581 - 14s/epoch - 24ms/step
Epoch 7/100
558/558 - 14s - loss: 1.5023 - accuracy: 0.3906 - val_loss: 1.5782 - val_accuracy: 0.3752 - 14s/epoch - 24ms/step
Epoch 8/100
558/558 - 13s - loss: 1.4554 - accuracy: 0.4089 - val_loss: 1.5288 - val_accuracy: 0.3979 - 13s/epoch - 24

In [ ]:
# Images fed into this model are 512 x 512 pixels with 3 channels

data_shape = (120,7,1)
#data_shape = (28,28,1)

# Set up the model

model = Sequential()

# Add convolutional layer with 3, 3 by 3 filters and a stride size of 1
# Set padding so that input size equals output size

model.add(Conv1D(4,3,input_shape=data_shape))
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 60, 5, 4)          16        
                                                                 
 flatten_3 (Flatten)         (None, 1200)              0         
                                                                 
 dense_9 (Dense)             (None, 10)                12010     
                                                                 
 dense_10 (Dense)            (None, 10)                110       
                                                                 
 dense_11 (Dense)            (None, 2)                 22        
                                                                 
Total params: 12,158
Trainable params: 12,158
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 학습

model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test) )

Epoch 1/10
564/564 [==============================] - 4s 6ms/step - loss: 0.3567 - accuracy: 0.8883 - val_loss: 0.3410 - val_accuracy: 0.8902
Epoch 2/10
564/564 [==============================] - 3s 6ms/step - loss: 0.3432 - accuracy: 0.8883 - val_loss: 0.3328 - val_accuracy: 0.8902
Epoch 3/10
564/564 [==============================] - 3s 6ms/step - loss: 0.3326 - accuracy: 0.8894 - val_loss: 0.3265 - val_accuracy: 0.8905
Epoch 4/10
564/564 [==============================] - 4s 6ms/step - loss: 0.3286 - accuracy: 0.8908 - val_loss: 0.3240 - val_accuracy: 0.8892
Epoch 5/10
564/564 [==============================] - 4s 6ms/step - loss: 0.3271 - accuracy: 0.8897 - val_loss: 0.3180 - val_accuracy: 0.8922
Epoch 6/10
564/564 [==============================] - 6s 10ms/step - loss: 0.3242 - accuracy: 0.8905 - val_loss: 0.3180 - val_accuracy: 0.8923
Epoch 7/10
564/564 [==============================] - 7s 12ms/step - loss: 0.3227 - accuracy: 0.8911 - val_loss: 0.3316 - val_accuracy: 0.8921
Epoc

In [49]:
y_test[3]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [52]:
result = model.predict(np.array([x_test[3]]))
print(result)
print(round(result[0][2],2) )
print(sum(result[0][:2]))
print(sum(result[0][2:11]))

[[9.7497785e-03 4.9183413e-01 3.9344984e-01 7.8155018e-02 3.3262772e-03
  2.3224082e-02 2.5867202e-04 2.2469571e-06 2.1050893e-08 5.3457794e-12
  1.9571680e-09]]
0.39
0.501583912409842
0.4984161624042898


In [47]:
model.save('cnn_prediction_xrp.h5')

In [48]:
%ls -al

total 28381
-rw------- 1 root root   69784 Dec 19 13:56 cnn.h5
drwx------ 4 root root    4096 Dec 19 09:41 cnn_prediction/
-rw------- 1 root root  185256 Dec 20 07:40 cnn_prediction_bora.h5
-rw------- 1 root root  185352 Dec 20 08:06 cnn_prediction_eos.h5
-rw------- 1 root root 4681048 Dec 26 15:14 cnn_prediction_sand.h5
-rw------- 1 root root  185256 Dec 20 07:55 cnn_prediction_xlm.h5
-rw------- 1 root root 4681048 Dec 26 16:04 cnn_prediction_xrp.h5
-rw------- 1 root root 3802325 Dec 20 07:35 KRW-BORA.csv
-rw------- 1 root root 3832322 Dec 20 08:03 KRW-EOS.csv
-rw------- 1 root root 3836665 Dec 20 07:10 KRW-SAND.csv
-rw------- 1 root root 3740209 Dec 20 07:46 KRW-XLM.csv
-rw------- 1 root root    1349 Dec 20 07:36 scaler_bora.pkl
-rw------- 1 root root    1349 Dec 20 08:03 scaler_eos.pkl
-rw------- 1 root root    1349 Dec 19 09:40 scaler.pkl
-rw------- 1 root root    1349 Dec 20 07:16 scaler_sand.pkl
-rw------- 1 root root    1349 Dec 20 07:47 scaler_xlm.pkl
-rw------- 1 root root    

In [ ]:
%pwd

'/content/drive/My Drive/스터디/코인 트레이드/data'

In [ ]:
from tensorflow import keras
model = keras.models.load_model('cnn_prediction')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60, 5, 4)          16        
                                                                 
 flatten (Flatten)           (None, 1200)              0         
                                                                 
 dense (Dense)               (None, 10)                12010     
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 12,158
Trainable params: 12,158
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%ls -al

total 3832
-rw------- 1 root root   69784 Dec 19 13:56 cnn.h5
drwx------ 4 root root    4096 Dec 19 09:41 cnn_prediction/
-rw------- 1 root root    1349 Dec 19 09:40 scaler.pkl
-rw------- 1 root root       0 Dec 19 07:57 train.csv
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [ ]:
data = x_test[0]

re_data = np.array([data,])


x_test.shape, data.shape, re_data.shape
#model.predict(data)

((12032, 60, 7, 1), (60, 7, 1), (1, 60, 7, 1))